In [1]:
import csv
from estnltk import Text

In [2]:
with open("morph/results/number_analysis_rules_with_examples.csv", "r") as fin:
    reader = csv.reader(fin)
    lines = []
    for line in reader:
        lines.append(line)

In [3]:
header = lines.pop(0)

In [4]:
# Koondkorpuses esinevad juhud
koond_lines = []
for line in lines:
    if line[5] != '':
        koond_lines.append(line)

In [5]:
correct_lines = []
correct_idx = []
correct_for_sven = []
correct_for_sven_idx = []

# Liigitame õigeteks need, mida vabamorf ilusti oletamata analüüsida oskab
for idx, line in enumerate(koond_lines):
    
    t = Text(line[5], disambiguate = False, guess = False, propername = False)

    if len(t.analysis) > 0:
        for a in t.analysis:

            for i in a:

                if i['partofspeech'] == 'N' and line[2] == 'N' and i['form'] == line[3]:
                    correct_lines.append(line)
                    correct_idx.append(idx)
                elif i['partofspeech'] == 'O' and line[2] == 'O' and i['form'] == line[3]:
                    correct_lines.append(line)
                    correct_idx.append(idx)
                    
                # Sveni soovil loeme õigeks ka n-ö unustatud punktiga järgarvude analüüsid    
                elif (i['partofspeech'] == 'O' or i['partofspeech'] == 'N') and i['form'] == line[3]:
                    if '\.' not in line[0]:
                        correct_for_sven.append(line)
                        correct_for_sven_idx.append(idx)
                    

In [6]:
other_lines = []
for idx, i in enumerate(koond_lines):
    if idx not in correct_idx and idx not in correct_for_sven_idx:
        other_lines.append(i)

In [7]:
# Loeme sobivaks ka juhud, kus inimene on kirjutanud välja korrektsest lõpust pikema tüki
almost_correct_lines = []
zero_endings = []
for idx, line in enumerate(other_lines):
    if line[1].endswith(line[4]):
        almost_correct_lines.append(line)
        
    # Aktsepteerime ka 0-lõpud (3-me jne)    
    elif line[4] == '0':
        zero_endings.append(line)

In [8]:
solved_lines = []
for line in correct_lines:
    line.append("ok")
    solved_lines.append(line)
for line in correct_for_sven:
    line.append("ok_for_sven")
    solved_lines.append(line)
for line in almost_correct_lines:
    line.append("good_enough")
    solved_lines.append(line)
for line in zero_endings:
    line.append("zero_ending")
    solved_lines.append(line)    

In [9]:
# Leiame koondkorpuse katmata näited
covered_examples = []
for line in solved_lines:
    covered_examples.append(line[5])
    
all_examples = []
for line in koond_lines:
    all_examples.append(line[5])    
    
not_covered_examples = set(all_examples) - set(covered_examples)    

In [10]:
# -se lõpud on ilmselt omadussõnalõpud ("nulline"->"nullise" jne).
# i-d ja a-d ja e-d võivad ka mis iganes olla, mitte tingimata käändelõpud
not_covered_examples

{'0-se',
 '215-se',
 '4i',
 '74e',
 '74se',
 '7500-e',
 '76se',
 '7i',
 '8i',
 '9000i',
 '9000se',
 '911-i',
 '91a',
 '92a',
 '93se',
 '9550se',
 '95se',
 '9600-i',
 '97-se',
 '9710i',
 '9800se',
 '98e',
 '98se',
 '99se',
 '9e',
 '9i'}

In [11]:
# Koondkorpuses mitte olevad näited:
not_koond_lines = []
for line in lines:
    if line[5] == '':
        not_koond_lines.append(line)

In [12]:
# Aktsepteerime korrektsest variandist pikemad
ok_no_example = []
not_ok = []
for line in not_koond_lines:
    if (len(line[1]) > len(line[4]) and line[4] != 0) or line[1] == line[4]:
        ok_no_example.append(line)  
    else:
        not_ok.append(line)

In [13]:
for line in ok_no_example:
    line.append("ok_no_example")
    solved_lines.append(line)
    

In [14]:
for line in lines:
    if line not in solved_lines:
        line.append("not_ok")
        solved_lines.append(line)

In [15]:
len(solved_lines)

9744

In [16]:
len(lines)

9744

In [17]:
with open("morph/results/number_analysis_rules_verified.csv", "w") as fout:
    writer = csv.writer(fout, quoting = csv.QUOTE_ALL)
    header.append('decision')
    writer.writerow(header)
    for row in solved_lines:
        writer.writerow(row)